In [1]:
# DataLoader
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

def loading_data(batch_size):
    training_set = datasets.FashionMNIST(root="data", train=True, transform=ToTensor(), download=True)
    test_set = datasets.FashionMNIST(root="data", train=False, transform=ToTensor(), download=True)
    train = DataLoader(training_set,batch_size=batch_size)
    test = DataLoader(test_set, batch_size=batch_size)
    return train,test

In [2]:
train,test = loading_data(256)

In [3]:
# Model

import torch
from torch import nn


def Device():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    return device


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 512),
            nn.Tanh(),
            nn.Linear(512, 512),
            nn.Tanh(),
            nn.Linear(512, 512),
            nn.Tanh(),
            nn.Linear(512, 512),
            nn.Linear(512, 10)
        )

    def forward(self, X):
        return self.net(X)


In [4]:
Model = Net().to(Device())
print(Model)

Net(
  (net): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=512, bias=True)
    (2): Tanh()
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): Tanh()
    (5): Linear(in_features=512, out_features=512, bias=True)
    (6): Tanh()
    (7): Linear(in_features=512, out_features=512, bias=True)
    (8): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
import torch.optim
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(Model.parameters(),lr=0.0825,momentum=0.0001)

In [6]:
len(train.dataset)

60000

In [7]:
def trainer(dataloader, model, optim, loss_fn):
    model.train()
    # train_mode
    size = len(dataloader.dataset)
    for batch,(X,y) in enumerate(dataloader):
        # computing loss
        X, y = X.to(Device()),y.to(Device())
        Loss = loss_fn(model(X), y)

        # SGD
        optim.zero_grad()
        Loss.backward()
        optim.step()

        if batch%100 == 0:
            loss,current = Loss.item(),batch * len(X)
            print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")

In [8]:
def confirmer(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    batches = len(dataloader)
    model.eval()
    test_loss,correct = 0, 0
    with torch.no_grad():
        for X,y in dataloader:
            X, y = X.to(Device()),y.to(Device())
            test_loss += loss_fn(model(X),y).item()
            correct += (model(X).argmax(1)==y).type(torch.float).sum().item()
    test_loss /= batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    trainer(train, Model, optimizer, loss_function)
    confirmer(test, Model, loss_function)
print("finished!")

Epoch 1
-------------------------------
loss:2.303764 [    0/60000]
loss:0.795618 [25600/60000]
loss:0.644000 [51200/60000]
Test Error: 
 Accuracy: 73.3%, Avg loss: 0.716740 

Epoch 2
-------------------------------
loss:0.715858 [    0/60000]
loss:0.522810 [25600/60000]
loss:0.520330 [51200/60000]
Test Error: 
 Accuracy: 77.8%, Avg loss: 0.607951 

Epoch 3
-------------------------------
loss:0.559346 [    0/60000]
loss:0.434130 [25600/60000]
loss:0.502800 [51200/60000]
Test Error: 
 Accuracy: 79.3%, Avg loss: 0.567990 

finished!


In [10]:
torch.save(Model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [11]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]
training_set = datasets.FashionMNIST(root="data", train=True, transform=ToTensor(), download=True)
test_set = datasets.FashionMNIST(root="data", train=False, transform=ToTensor(), download=True)
cnt = 0
for i in range(1000):
    import  random
    j = random.randint(0,5000)
    Model.eval()
    x, y = training_set[j][0], test_set[j][1]
    x = x.cuda()
    with torch.no_grad():
        pred = Model(x)
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        if predicted == actual: cnt+=1
        print(f'{i+1}: Predicted: "{predicted}", Actual: "{actual}"')
print(cnt)

1: Predicted: "Pullover", Actual: "Sneaker"
2: Predicted: "T-shirt/top", Actual: "Shirt"
3: Predicted: "Sneaker", Actual: "Sandal"
4: Predicted: "Trouser", Actual: "Coat"
5: Predicted: "Sandal", Actual: "Shirt"
6: Predicted: "Shirt", Actual: "Bag"
7: Predicted: "Trouser", Actual: "T-shirt/top"
8: Predicted: "Trouser", Actual: "Shirt"
9: Predicted: "Sneaker", Actual: "Sneaker"
10: Predicted: "Sandal", Actual: "Coat"
11: Predicted: "Sandal", Actual: "Bag"
12: Predicted: "T-shirt/top", Actual: "Shirt"
13: Predicted: "T-shirt/top", Actual: "Ankle boot"
14: Predicted: "Bag", Actual: "Pullover"
15: Predicted: "Ankle boot", Actual: "Pullover"
16: Predicted: "Dress", Actual: "Coat"
17: Predicted: "Sneaker", Actual: "Trouser"
18: Predicted: "Bag", Actual: "Sandal"
19: Predicted: "T-shirt/top", Actual: "Sneaker"
20: Predicted: "Coat", Actual: "Shirt"
21: Predicted: "Bag", Actual: "Sneaker"
22: Predicted: "Bag", Actual: "Coat"
23: Predicted: "Coat", Actual: "T-shirt/top"
24: Predicted: "Ankle boo